# Importation des Librairies

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim

## 1. Préparation des données

1.1 Importer les données Train et Test

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path = '/content/drive/MyDrive/Data_Science/train_clean.csv'
test_path = '/content/drive/My Drive/Data_Science/test_clean.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

1.2 Depuis le Dataframe train, charger les features d'apprentissage dans un array numpy X_alltrain, et les labels (données à prévoir) dans un array numpy y_alltrain

In [4]:
X_alltrain = train_df.drop(columns=['Survived']).to_numpy()
y_alltrain = train_df['Survived'].to_numpy()

1.3 Séparer les features et les labels en deux parties (train et dev), en attribuant 10% des exemples aux données de dev. afficher les nombres de lignes et de colonnes pour les 4 arrays.

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X_alltrain, y_alltrain, test_size=0.1, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_dev shape: {X_dev.shape}")
print(f"y_dev shape: {y_dev.shape}")

X_train shape: (801, 8)
y_train shape: (801,)
X_dev shape: (90, 8)
y_dev shape: (90,)


1.4 Afficher les 10 premières lignes de features et les 10 premiers labels.

In [6]:
print("10 premieres lignes des features:")
print(X_train[:10])
print("10 premiers labels:")
print(y_train[:10])

10 premieres lignes des features:
[[166   3   1   0   2   0   0   4]
 [542   3   0   0   3   0   0   2]
 [626   1   1   3   3   0   1   1]
 [389   3   1   2   0   2   1   1]
 [ 77   3   1   0   0   0   1   1]
 [282   3   1   1   0   0   1   1]
 [713   1   1   2   3   0   0   1]
 [339   3   1   2   1   0   1   1]
 [328   2   0   2   1   0   1   3]
 [322   3   1   1   0   0   1   1]]
10 premiers labels:
[1 0 0 0 0 0 1 1 1 0]


## 2. Définition du réseau de neurone

2.1 Définir et instancier une classe Titanic Model avec les caractéristiques suivantes :
- deux  couches cachées de 50 neurones.
- deux classes en sortie : Survivant ou non
- Des fonctions d'activation RELU
- un dropout paramétrable pour les 2 couches cachées

In [7]:
class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_prob):
        super(TitanicModel, self).__init__()
        self.hidden1 = nn.Linear(input_size, 50)
        self.hidden2 = nn.Linear(50, 50)
        self.output = nn.Linear(50, 2)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.dropout(x)
        x = self.output(x)
        return x

input_size = X_train.shape[1]
dropout_prob = 0.5
model = TitanicModel(input_size, dropout_prob)

2.2 : Définir des paramètres de nombre d'epochs (50) et de learning_rate (0.01)

In [8]:
epochs = 50
learning_rate = 0.01

2.3 définir la taille du minibatch à 50. En déduire le nombre de boucle pour chaque epoch.

In [9]:
batch_size = 50
n_batches = len(X_train) // batch_size
print(f"Nombre de boucles par epoch: {n_batches}")

Nombre de boucles par epoch: 16


2.4 Définir un fonction de coût de type CrossEntropy

In [10]:
criterion = nn.CrossEntropyLoss()

2.5. Définir un optimizer de type Adam, sans oublier le learning rate

In [11]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


## 3. Apprentissage

3.1 exécuter l'apprentissage du modèle.
- Créer une boucle sur les epochs, qui contient elel-même une boucle sur les minibatchs.
- A chaque nouvelle itération sur les epochs, mélanger les données avec la méthode shuffle.
- Tous les 5 epochs afficher la valeur de la fonction de cout

In [20]:
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch + 1) % 5 == 0:
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Perte moyenne: {avg_loss:.4f}")


Epoch 5/50, Perte moyenne: 0.6519
Epoch 10/50, Perte moyenne: 0.6539
Epoch 15/50, Perte moyenne: 0.6483
Epoch 20/50, Perte moyenne: 0.6729
Epoch 25/50, Perte moyenne: 0.6827
Epoch 30/50, Perte moyenne: 0.6507
Epoch 35/50, Perte moyenne: 0.6505
Epoch 40/50, Perte moyenne: 0.6739
Epoch 45/50, Perte moyenne: 0.6509
Epoch 50/50, Perte moyenne: 0.6727


3.2 Calculer la précision de la prévision sur les données dev

In [21]:
from sklearn.metrics import accuracy_score

X_dev_tensor = torch.tensor(X_dev, dtype=torch.float32)
y_dev_tensor = torch.tensor(y_dev, dtype=torch.long)

model.eval()
with torch.no_grad():
  logits = model(X_dev_tensor)
  predictions = torch.argmax(logits, dim=1)

accuracy = accuracy_score(y_dev, predictions.numpy())
print(f"Précision : {accuracy:.4f}")


Précision : 0.5889


3.3 Calculer prévisions sur les données de tests

In [25]:
X_test_tensor = torch.tensor(test_df.values, dtype=torch.float32)
model.eval()
with torch.no_grad():
  logits_test = model(X_test_tensor)
  predictions_test = torch.argmax(logits_test, dim=1)
predictions_test = predictions_test.numpy()

3.4 Générer le fichier résultat et l'envoyer sur kaggle
Quel est votre score et votre classement ?

In [27]:
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": predictions_test })

submission_file = "submission.csv"
submission.to_csv(submission_file, index=False)

3.5 (Optionnel) Exécuter une Cross Validationd ans une boucle pour trouver les meilleures valeurs de learning rate, de keep_prob et de nombre d'epochs.

Cross_Validation avec skkearn : https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators